# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

#base url needed for the api call
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.DataFrame(pd.read_csv("output_data/cities.csv"))
cities_df["temp (F)"] = (cities_df["temp"]  * 1.8) + 32
cities_df["temp_max (F)"] = (cities_df["temp_max"]  * 1.8) + 32
cities_df.head()

,city,lat,lon,temp,humidity,clouds,windspeed,temp_max,temp (F),temp_max (F)
0,vaitape,-16.52,-151.75,28.25,73,17,3.88,28.25,82.85,82.850
1,gat,31.61,34.76,14.00,93,0,1.50,16.67,57.20,62.006
2,maniitsoq,65.42,-52.90,-17.15,72,4,6.20,-17.15,1.13,1.130
3,kapaa,22.08,-159.32,21.85,94,90,11.80,23.00,71.33,73.400
4,atuona,-9.80,-139.03,27.85,68,14,4.02,27.85,82.13,82.130


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
coordinates = []
humidity = []
for i in range(len(cities_df)):
    coordinates.append([cities_df["lat"][i], cities_df["lon"][i]])
    humidity.append(cities_df["humidity"][i])


fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1.5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
matching_hotels_df = cities_df[ (cities_df['temp_max (F)']>70) & (cities_df['temp_max (F)']<80) & (cities_df['windspeed']<10)  & (cities_df['clouds']==0)]



### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
city_list = matching_hotels_df["city"].tolist()
lat_list = matching_hotels_df["lat"].tolist()
lon_list = matching_hotels_df["lon"].tolist()
humidity_list = matching_hotels_df["humidity"].tolist()
coords = list(zip(lat_list, lon_list, city_list, humidity_list))
coords_df = pd.DataFrame(coords)
coordinates_df = coords_df.rename(columns={0: "lat", 1: "lon", 2: "city", 3: "humidity"})
coordinates_df["Hotel_Coordinates"] = "" 

for i in range(len(coordinates_df)):
    coordinates_df["Hotel_Coordinates"][i] = "'" + str(coordinates_df["lat"][i]) + "','" + str(coordinates_df["lon"][i]) +"'"
    #str(coordinates_df["lat"][i]) + str("', '") + str(coordinates_df["lon"][i])

hotel_coordinates = []
for coord in coords:
    hotel_coordinates.append(str(coord[0]) + ', ' + str(coord[1]))


coordinates_df = coordinates_df.reset_index()
del coordinates_df["index"]
coordinates_df["Hotel_Name"] = "Excluded"
coordinates_df

C:\Users\busha\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,lat,lon,city,humidity,Hotel_Coordinates,Hotel_Name
0,-8.47,118.72,bima,93,"'-8.47','118.72'",Excluded
1,-20.28,44.28,morondava,95,"'-20.28','44.28'",Excluded
2,-38.55,-58.74,necochea,73,"'-38.55','-58.74'",Excluded
3,-26.02,22.40,tsabong,29,"'-26.02','22.4'",Excluded
4,-33.70,-53.46,chuy,58,"'-33.7','-53.46'",Excluded
5,5.40,26.49,obo,39,"'5.4','26.49'",Excluded
6,24.09,32.91,aswan,34,"'24.09','32.91'",Excluded
7,-21.17,27.51,francistown,61,"'-21.17','27.51'",Excluded
8,11.00,122.67,dingle,89,"'11.0','122.67'",Excluded
9,10.07,34.53,asosa,18,"'10.07','34.53'",Excluded


In [6]:
hotels_matching_parameters = []
i = 0
for hotel_coordinate in hotel_coordinates:
    
    try:
        parameters = {
            "location": hotel_coordinate,
            "keyword": "hotel",
            "radius":  "5000",
            "key": g_key
        }

        response = requests.get(base_url, params = parameters)
        places_data = response.json()

        hotels_matching_parameters.append(places_data["results"][0]["name"])
        coordinates_df["Hotel_Name"][i] = places_data["results"][0]["name"]
        i += 1
    except (KeyError, ValueError, IndexError):
            print("Hotel not found based on parameters - skipping...")
            i += 1
hotels_temp_df = coordinates_df.loc[coordinates_df["Hotel_Name"] != "Excluded"]
hotels_df = hotels_temp_df.reset_index()

C:\Users\busha\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Hotel not found based on parameters - skipping...
Hotel not found based on parameters - skipping...
Hotel not found based on parameters - skipping...


In [7]:
locations = []

for i in range(len(hotels_df)):
    #print(hotels_df['lat'][i])
    locations.append([hotels_df['lat'][i], hotels_df['lon'][i]])

    
humidity_list = hotels_df["humidity"].astype(float).to_list()

# Plot Heatmap
hfig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_list, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
hfig.add_layer(heat_layer)

# Display figure
hfig


Figure(layout=FigureLayout(height='420px'))

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>Not listed</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
locations = hotels_df[["lat", "lon"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {Hotel}" for Hotel in hotels_df["Hotel_Name"]])
hfig.add_layer(markers)

# Display Map
hfig

Figure(layout=FigureLayout(height='420px'))